In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)

In [2]:
import random

def random_mul_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a * val_b
    str_c = str(val_c)
    str_c = (2*length - len(str_c)) * '0' + str_c
    return a + b + [int(d) for d in str_c]

for i in range(10):
    print (random_mul_instance(3))

[0, 6, 2, 5, 3, 6, 0, 3, 3, 2, 3, 2]
[4, 9, 5, 3, 7, 8, 1, 8, 7, 1, 1, 0]
[6, 4, 4, 3, 1, 4, 2, 0, 2, 2, 1, 6]
[3, 5, 9, 1, 1, 9, 0, 4, 2, 7, 2, 1]
[3, 5, 0, 1, 4, 8, 0, 5, 1, 8, 0, 0]
[0, 0, 2, 9, 3, 7, 0, 0, 1, 8, 7, 4]
[8, 4, 8, 2, 3, 7, 2, 0, 0, 9, 7, 6]
[9, 9, 9, 7, 0, 6, 7, 0, 5, 2, 9, 4]
[7, 4, 5, 1, 3, 6, 1, 0, 1, 3, 2, 0]
[1, 0, 1, 0, 4, 2, 0, 0, 4, 2, 4, 2]


In [3]:
class MulDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 4 * self.length - 1

    def __getitem__(self, idx):
        while True:
            rai = random_mul_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [4]:
train_dataset = MulDataset('train')
test_dataset = MulDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([2, 9, 3, 4, 8, 0, 1, 4, 0, 6, 4])
2 -1
9 -1
3 -1
4 -1
8 -1
0 1
1 4
4 0
0 6
6 4
4 0


In [5]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [6]:
print (model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [11]:

# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 10000
train_config.num_workers = 0
# train_config.batch_size = 32
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [15]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 79.81ms; iter 0: train loss 1.09415
iter_dt 67.02ms; iter 100: train loss 1.10155
iter_dt 67.31ms; iter 200: train loss 1.12475
iter_dt 63.87ms; iter 300: train loss 1.09395
iter_dt 57.33ms; iter 400: train loss 1.08058
iter_dt 60.40ms; iter 500: train loss 1.07905
iter_dt 76.66ms; iter 600: train loss 1.04403
iter_dt 61.03ms; iter 700: train loss 1.01314
iter_dt 64.20ms; iter 800: train loss 1.02203
iter_dt 70.65ms; iter 900: train loss 1.05949
iter_dt 57.30ms; iter 1000: train loss 1.06026
iter_dt 66.95ms; iter 1100: train loss 1.06293
iter_dt 65.30ms; iter 1200: train loss 1.09616
iter_dt 56.63ms; iter 1300: train loss 1.08743
iter_dt 63.58ms; iter 1400: train loss 1.03016
iter_dt 57.76ms; iter 1500: train loss 1.09013
iter_dt 68.93ms; iter 1600: train loss 1.06123
iter_dt 61.23ms; iter 1700: train loss 1.01014
iter_dt 64.99ms; iter 1800: train loss 1.02373
iter_dt 60.56ms; iter 1900: train loss 0.98242
iter_dt 61.72ms; iter 2000: train loss 1.04382
iter_dt 57.59ms; iter 210

In [16]:
# now let's perform some evaluation
model.eval()
None

In [17]:
def eval_add_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -2*n:]
        
        cat = model.generate(inp, 2*n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -2*n:]      
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_add_split(trainer, 'train', max_batches=50)
    test_score  = eval_add_split(trainer, 'test',  max_batches=50)

train final score: 328/10000 = 3.28% correct
test final score: 337/10000 = 3.37% correct
